In [ ]:
!pip install langchain-pinecone --quiet
!pip install langchain --quiet
!pip install openai --quiet
!pip install pinecone-client --quiet
!pip install langchain_openai --quiet
!pip install langchain-community --quiet
!pip install langchain_upstage --quiet
!pip install langchain_core --quiet

In [ ]:
!pip install langchain_text_splitters --quiet

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain_upstage import UpstageEmbeddings
from langchain.prompts import PromptTemplate
from langchain_upstage import ChatUpstage
from openai import OpenAI
import pinecone
from langchain_openai import ChatOpenAI
import os
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
UPSTAGE_API_KEY = "your-api-key"
PINECONE_API_KEY = "4cf70d26-24f1-4438-8216-2e2b193835e3"
INDEX_NAME = "greeniche"
PINECONE_ENV = 'gcp-starter'

# 복귀

In [ ]:
prompt = PromptTemplate(
    template="""다음 지시에 따라 질문에 대한 답변을 작성하세요.

### 지시:
주어진 문서를 바탕으로 질문에 답하세요. 문서의 정보만을 사용하고, 문서에 없는 정보는 사용하지 마세요.

### 문서:
{context}

### 질문:
{question}

### 답변:
""",
    input_variables=["context", "question"]
)

In [ ]:
embeddings = UpstageEmbeddings(model="solar-embedding-1-large-query",
                               upstage_api_key=UPSTAGE_API_KEY)

In [ ]:
db = PineconeVectorStore(
    pinecone_api_key=PINECONE_API_KEY,
    index_name=INDEX_NAME,
    embedding=embeddings,
    text_key='chunk'
)

In [ ]:
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 3, "score_threshold": 0.5})

In [ ]:
llm = ChatOpenAI(
    api_key=UPSTAGE_API_KEY,
    model_name='solar-pro',
    temperature=0.0)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro", max_tokens = 2000),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

In [ ]:
query = "기후변화 대응 위 기술 혁신 확산과 관련된 ESRS, GRI 인덱스는 뭔지 짧게 요약해줘"
result = qa_chain.invoke({"query": query})['result']
print(result)

ESRS 정보: 엔디터가 기후변화 대응에 관한 정책, 특히 엔디터의 혁신 확산과 관련된 정책을 설명해야 합니다. ESRS 또한 기후변화에 대한 행동 및 할당된 자원에 대한 설명을 요구합니다.

GRI 정보: 조직은 기후변화에 대한 리스크와 기회, 특히 기후 변화에 따른 재무적 영향에 대한 설명, 이러한 리스크와 기회에 대한 설명, 영향에 대한 설명, 대응 조치 전 재무적 영향, 이러한 리스크와 기회를 관리하는 방법, 관리 관련 비용에 대한 설명을 제공해야 합니다.

요약하면, ESRS 및 GRI 모두 조직이 기후변화 대응, 특히 기후 변화 대응을 위한 기술 혁신의 확산과 관련된 정책, 행동, 리스크와 기회, 재무 영향에 대한 명확한 설명을 제공해야 합니다.


# Demo

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00


In [ ]:
%%writefile documentparser.py
import requests
import streamlit as st

UPSTAGE_API_KEY = "up_ZL8Fz6IcyZjyREGqzoV0Q4IOSVdho"

def parse_document(uploaded_file):
    url = "https://api.upstage.ai/v1/document-ai/document-parse"
    headers = {"Authorization": f"Bearer {UPSTAGE_API_KEY}"}

    files = {"document": uploaded_file.getvalue()}

    try:
        response = requests.post(url, headers=headers, files=files)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        data = response.json()
        if 'content' in data and 'text' in data['content']:
            return data['content']['text']
        else:
            st.error("API response does not contain expected 'content.text' field")
            st.json(data)  # Display the full response for debugging
            return None
    except requests.exceptions.RequestException as e:
        st.error(f"Error parsing document: {str(e)}")
        return None

Overwriting documentparser.py


In [ ]:
%%writefile app.py
import streamlit as st
from documentparser import parse_document
from ragmodel import initialize_rag, ask_question
from langchain.memory import ConversationBufferMemory

st.title("ESG converting chatbot")
st.write("Upload a document and ask questions about it!")

# 사이드바에 제목 추가
st.sidebar.title("컨텍스트 정보")

# 추가 정보나 컨트롤 추가
with st.sidebar:
    st.markdown("---")
    st.subheader("필터")
    selected_option = st.selectbox("정보 유형 선택", ["모두", "ESRS", "GRI"])

# 세션 상태 초기화
if 'parsed_text' not in st.session_state:
    st.session_state.parsed_text = None
if 'qa_chain' not in st.session_state:
    st.session_state.qa_chain = None

# 메모리 상태 초기화
if 'memory' not in st.session_state:
    st.session_state.memory = ConversationBufferMemory()

# 메모리 초기화 버튼
if st.sidebar.button('메모리 초기화'):
    st.session_state.memory = ConversationBufferMemory()
    st.success('메모리가 초기화되었습니다.')

# 현재 메모리 상태 표시
current_buffer = st.session_state.memory.buffer
st.sidebar.write('현재 메모리 상태:', current_buffer)

uploaded_file = st.file_uploader("Upload a document (.txt, .md, .pdf)", type=("txt", "md", "pdf"))

if uploaded_file and st.session_state.parsed_text is None:
    with st.spinner("문서 분석 중 ..."):
        st.session_state.parsed_text = parse_document(uploaded_file)
    st.success("Document uploaded and parsed successfully!")

    # RAG 모델 초기화 (문서가 파싱된 후 한 번만 실행)
    st.session_state.qa_chain = initialize_rag(st.session_state.parsed_text)

if st.session_state.parsed_text is not None:
    question = st.text_area("질문을 입력하세요:", placeholder="궁금하신 ESRS나 GRI 지표에 대해 질문하세요!")

    if question:
        with st.spinner("Generating answer..."):
            response = ask_question(st.session_state.qa_chain, question)

        st.markdown("### Answer:")
        st.markdown(response)

        # 메모리에 대화 추가
        st.session_state.memory.save_context({"input": question}, {"output": response})

        if "messages" not in st.session_state:
            st.session_state.messages = []
        st.session_state.messages.append({"role": "user", "content": question})
        st.session_state.messages.append({"role": "assistant", "content": response})

if "messages" in st.session_state:
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

Overwriting app.py


In [ ]:
%%writefile ragmodel.py
import streamlit as st
from langchain_pinecone import PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter

UPSTAGE_API_KEY = "up_ZL8Fz6IcyZjyREGqzoV0Q4IOSVdho"
PINECONE_API_KEY = "4cf70d26-24f1-4438-8216-2e2b193835e3"
INDEX_NAME = "upstage"
PINECONE_ENV = 'gcp-starter'
@st.cache_resource
def initialize_rag(parsed_text):
    prompt = PromptTemplate(
        template="""다음 지시에 따라 질문에 대한 답변을 작성하세요.

    ### 지시:
    주어진 문서를 바탕으로 질문에 답하세요. 문서의 정보만을 사용하고, 문서에 없는 정보는 사용하지 마세요.

    ### 문서:
    {context}

    ### 질문:
    {question}

    ### 답변:
    """,
        input_variables=["context", "question"]
    )

    embeddings = UpstageEmbeddings(model="solar-embedding-1-large-query",
                                   upstage_api_key=UPSTAGE_API_KEY)

    # 텍스트 분할
    text_splitter = CharacterTextSplitter(chunk_size=150, chunk_overlap=0)
    texts = text_splitter.split_text(parsed_text)

    # 문서 생성
    documents = [Document(page_content=t) for t in texts]

    # Pinecone에 문서 저장
    db = PineconeVectorStore(
        pinecone_api_key=PINECONE_API_KEY,
        index_name=INDEX_NAME,
        embedding=embeddings,
        text_key='chunk'
    )

    qa_chain = RetrievalQA.from_chain_type(
    llm=ChatUpstage(api_key=UPSTAGE_API_KEY, model="solar-pro", max_tokens=1000),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)


    return qa_chain



def ask_question(qa_chain, question):
    return qa_chain({"query": question})["result"]

Overwriting ragmodel.py


In [ ]:
!npm install localtunnel


up to date, audited 23 packages in 518ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
import urllib
print("Password/Endpoint IP:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Endpoint IP: 34.80.215.241


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

your url is: https://tough-sheep-prove.loca.lt
